In [1]:
INP_DIR = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'
conditions = ['spinal_canal_stenosis', 'left_neural_foraminal_narrowing', 'right_neural_foraminal_narrowing',
              'left_subarticular_stenosis', 'right_subarticular_stenosis']
levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity = ['Normal/Mild', 'Moderate', 'Severe']
TARGETS = [c+'_'+l+'_'+s for c in conditions for l in levels for s in severity]
SIZE = 256

In [2]:
import pandas as pd
from glob import glob

series_description = 'Axial T2'

In [5]:
df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')
df = df[df['series_description'] == series_description]
df

,study_id,series_id,series_description
2,4003253,2448190387,Axial T2
3,4646740,3201256954,Axial T2
7,7143189,1951927562,Axial T2
11,8785691,2406919186,Axial T2
12,10728036,142859125,Axial T2
...,...,...,...
6279,4282019580,121051321,Axial T2
6284,4283570761,3321662903,Axial T2
6287,4284048608,3891595425,Axial T2
6290,4287160193,1820446240,Axial T2


In [6]:
import os
import sys
import joblib


import numpy as np
import pandas as pd
from glob import glob
# import dicomsdl
import torch
import torch.nn as nn
import torch.nn.functional as F
import pydicom

from tqdm import tqdm

import cv2
from timeit import default_timer as timer



def dicomsdl_to_numpy_image(ds, index=0):
    info = ds.getPixelDataInfo()
    if info['SamplesPerPixel'] != 1:
        raise RuntimeError('SamplesPerPixel != 1')  # number of separate planes in this image
    shape = [info['Rows'], info['Cols']]
    dtype = info['dtype']
    outarr = np.empty(shape, dtype=dtype)
    ds.copyFrameData(index, outarr)
    return outarr


def convert_to_8bit(x):
    lower, upper = np.percentile(x, (1, 99))
    x = np.clip(x, lower, upper)
    x = x - np.min(x)
    x = x / np.max(x) 
    return (x * 255).astype("uint8")


def load_dicomsdl_dir(series_id, dcm_dir, slice_range=None):
    dcm_file = sorted(glob(f'{dcm_dir}/*.dcm'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
    dicoms = [pydicom.dcmread(f) for f in dcm_file]
    plane = 'axial'
    plane = {"sagittal": 0, "coronal": 1, "axial": 2}[plane.lower()]
    positions = np.asarray([float(d.ImagePositionPatient[plane]) for d in dicoms])
    reverse_sort = False
    idx = np.argsort(-positions if reverse_sort else positions)
    ipp = np.asarray([d.ImagePositionPatient for d in dicoms]).astype("float")[idx]
    image = [d.pixel_array.astype("float32") for d in dicoms]
    x0 = np.mean([im.shape[0] for im in image])
    y0 = np.mean([im.shape[1] for im in image])
    image = np.stack([cv2.resize(im, (SIZE, SIZE),interpolation=cv2.INTER_CUBIC) for im in image])
    image = image[idx]
    image = convert_to_8bit(image)
    return image, x0, y0

def prepare_images(study_id, series_id, to = 'preprocessed'):
    if not os.path.isdir(to):
        os.mkdir(to)
    dcm_dir = f'{INP_DIR}/train_images/{study_id}/{series_id}'
    image, x, y = load_dicomsdl_dir(series_id, dcm_dir, slice_range=None)
    np.save(f'{to}/{study_id}_{series_id}.npy', image)
    return x, y

In [7]:
to = 'preprocessed'
orig_size = {}
nbr_processed = 0
for ind, row in df[['study_id', 'series_id']].drop_duplicates().iterrows():
    orig_size[row.series_id] = prepare_images(row.study_id, row.series_id, to)
    nbr_processed += 1
    if nbr_processed % 50 == 0:
        print(nbr_processed)
import joblib
joblib.dump(orig_size, 'orig_size.pkl')

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300


['orig_size.pkl']